# CancellationToken

## CancellationTokenSource
Pozrite si nasledujúce kódy na `CancellationTokenSource` (https://learn.microsoft.com/en-us/dotnet/api/system.threading.cancellationtokensource):

### Metóda Cancel()

In [24]:
using System.Threading;

public static async Task DoWorkAsync(CancellationToken token)
{
    for (int i = 0; i < 10; i++)
    {
        token.ThrowIfCancellationRequested(); // Vyhodí výnimku OperationCanceledException, ak bol token zrušený

        Console.WriteLine($"Beží asynchrónna úloha... {i + 1}");
        await Task.Delay(1000); // Simulácia asynchrónneho čakania 1000 ms
    }
}

CancellationTokenSource cts = new CancellationTokenSource();
Task task = DoWorkAsync(cts.Token);

// Zrušenie po 3 sekundách
await Task.Delay(3000);
cts.Cancel();

try
{
    await task;
}
catch (OperationCanceledException)
{
    Console.WriteLine("Asynchrónna úloha bola zrušená.");
}


### Metóda CancelAfter()

In [ ]:
using System.Threading;

CancellationTokenSource cts = new CancellationTokenSource();

// Nastavenie časového limitu 2 sekundy
cts.CancelAfter(TimeSpan.FromSeconds(2));

try
{
    await Task.Delay(5000, cts.Token); // Úloha čaká 5 sekúnd, ale token sa zruší po 2 sekundách
}
catch (OperationCanceledException)
{
    Console.WriteLine("Úloha bola zrušená z dôvodu časového limitu.");
}

Napíšte asynchrónnu verziu tejto metódy, ktorá bude vracať reťazec `"Ahoj, async svet!"`:

In [ ]:
using System;
using System.IO;
using System.Net.Http;
using System.Threading;

public async Task DownloadFileAsync(string url, string filePath, CancellationToken token)
{
    HttpClient _httpClient = new();
    using HttpResponseMessage response = await _httpClient.GetAsync(url, HttpCompletionOption.ResponseHeadersRead, token);
    response.EnsureSuccessStatusCode();

    using var fileStream = new FileStream(filePath, FileMode.Create, FileAccess.Write, FileShare.None);
    using var httpStream = await response.Content.ReadAsStreamAsync(token);

    byte[] buffer = new byte[8192];
    int bytesRead;
    while ((bytesRead = await httpStream.ReadAsync(buffer, 0, buffer.Length, token)) > 0)
    {
        await fileStream.WriteAsync(buffer, 0, bytesRead, token);
    }

    Console.WriteLine("Súbor úspešne stiahnutý.");
}

CancellationTokenSource downloadFileCancellationTokenSource = new();
//downloadFileCancellationTokenSource.CancelAfter(5000);

//await DownloadFileAsync("https://releases.ubuntu.com/24.04.1/ubuntu-24.04.1-desktop-amd64.iso", "ubuntu-24.04.1-desktop-amd64.iso", downloadFileCancellationTokenSource.Token);

## Úloha 7

Výpočet PI pomocou Monte Carlo metódy:

In [ ]:
using System.Threading;

public async Task<double> CalculatePiAsync(int iterations)
{
    int insideCircle = 0;

    await Task.Run(() =>
    {
        Random random = new Random();
        for (int i = 0; i < iterations; i++)
        {
            double x = random.NextDouble();
            double y = random.NextDouble();

            if (x * x + y * y <= 1)
            {
                insideCircle++;
            }
        }
    });

    // Výpočet Pi
    return 4.0 * insideCircle / iterations;
}

Console.WriteLine("Začiatok výpočtu Pi...");
double pi = await CalculatePiAsync(100_000_000);
Console.WriteLine($"Približná hodnota pi: {pi}");


Pridajte do metódy `CalculatePiAsync` parameter `CancellationToken token`, pomocou ktorého bude možné zrušiť výpočet Pi 

In [ ]:
using System.Threading;

// TODO: Skopírujte kód vyššie a pridajte podporu pre CancellationToken token


var cts = new CancellationTokenSource();

try
{
    Console.WriteLine("Začiatok výpočtu pi...");
    double pi = await CalculatePiAsync(100_000_000, cts.Token);
    Console.WriteLine($"Približná hodnota pi: {pi}");
}
catch (OperationCanceledException)
{
    Console.WriteLine("Výpočet bol zrušený.");
}